In [42]:
# Load packages
# Established packages
import os
import numpy as np
import pandas as pd
import geopandas as gp
import datetime
import math
import xarray as xr # labels in the form of dimensions, coordinates and attributes
import matplotlib.pyplot as plt
import zarr

# Custom package covid19model
from covid19model.models import models
from covid19model.data import model_parameters, mobility #, sciensano, google
from covid19model.visualization.output import population_status, infected, show_map, show_graphs
from covid19model.models.time_dependant_parameter_fncs import lockdown_func, mobility_update_func, policies_wave1_4prev

# Moving average for visualisation
from covid19model.visualization.utils import moving_avg
from covid19model.models.utils import read_pops, read_coordinates_nis, initial_state

# Download function for complete calibration
# covid19model.optimization.run_optimization import full_calibration_wave1, full_calibration_wave2


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
# This may be useful because the `covid19model` package is under construction
%autoreload 2

# Spatial stratification
agg = 'arr'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = model_parameters.get_interaction_matrices(decades=False, spatial=agg)

In [44]:
# Check shape of initN
initN.shape

(43, 10)

In [45]:
# Check if the contact matrices sum up to (approximately) the correct values:
(Nc_home + Nc_work + Nc_schools + Nc_transport + Nc_leisure + Nc_others) - Nc_total

array([[ 7.10542736e-15,  1.11022302e-16,  1.11022302e-16,
         1.55431223e-15, -6.66133815e-15, -1.99840144e-15,
         9.99200722e-16,  5.55111512e-17,  5.82867088e-16,
        -3.81639165e-17],
       [-2.44249065e-15,  3.73034936e-14,  6.21724894e-15,
         0.00000000e+00, -1.77635684e-15, -2.22044605e-16,
         1.11022302e-16,  5.55111512e-17,  5.55111512e-17,
         9.71445147e-17],
       [-1.11022302e-15,  3.10862447e-15, -9.76996262e-15,
        -7.99360578e-15,  3.55271368e-15,  3.10862447e-15,
        -3.33066907e-16,  2.22044605e-16,  2.77555756e-17,
        -5.27355937e-16],
       [-1.77635684e-15, -4.66293670e-15, -8.43769499e-15,
        -2.66453526e-15,  8.88178420e-16,  2.66453526e-15,
         1.33226763e-15,  0.00000000e+00, -3.88578059e-16,
         4.44089210e-16],
       [ 0.00000000e+00,  2.22044605e-16,  1.77635684e-15,
        -4.44089210e-16,  7.99360578e-15, -3.55271368e-15,
         3.99680289e-15, -1.11022302e-16,  7.77156117e-16,
         1.